# Cierres (closures)

Vamos a ver qué hace el siguiente código:

In [89]:
def primera(x):
    def segunda(y):
        return x + y
    return segunda

En este código tenemos:

1. Una función anidada (una función dentro de otra)
2. La función anidada hace referencia a un valor definido en la función envolvente
3. La función envolvente devuelve la función anidada

¿Qué sucede si ejecutamos `primera`?

In [90]:
primera(1)

<function __main__.primera.<locals>.segunda>

Tenemos una función. Hemos construido una función que devuelve otra función. Vamos a darle un nombre a esa función devuelta y la vamos a ejecutar:

In [91]:
f = primera(1)
f

<function __main__.primera.<locals>.segunda>

In [10]:
f(2)

3

Por lo tanto lo que hemos hecho es crear una función que se llama `primera` que recibe un parámetro `x`. En el cuerpo de `primera` creamos una función interna que se llama `segunda` y que espera recibir otro parámetro, `y`. La función interna devuelve la suma de `x` e `y`, y la función externa devuelve la función interna.

¿Pero cómo es que `segunda` tiene acceso a `x`? ¿No lanza Python una excepción?

Veamos qué hace Python cuando llega a ese punto de código:

+ Mira si `x` está en el diccionario de `locals()`. `locals()` devuelve las variables definidas en el ámbito más cercano, que en este caso es el de `segunda`. Si encuentra, entonces el valor de `x` es el valor asociado a `x` en `locals()`:

```python
if x in locals():
    return locals()[x]
```
        
+ Mira si `x` está en el diccionario de `globals()`. `globals()` devuelve las variables definidas a nivel de módulo, es decir las variables que no están definidas dentro de un ámbito de clases o función. Si encuentra, entonces el valor de `x` es el valor asociado a `x` en `globals()`:

```python
if x in locals():
    return locals()[x]
```

+ Mira si `x` está en el diccionario de `__builtins__.__dict__`. `__builtins__.__dict__` contiene aquellas variables definidas en el módulo `builtin`y son importadas automáticamente cuando se ejecuta Python). Si encuentra, entonces el valor de `x` es el valor asociado a `x` en `__builtins__.__dict__`:

```python
if x in __builtins__.__dict__:
return __builtins__.__dict__[x]
```
        
+ Si no se cumple nada de lo anterior, lanza una excepción `NameError`.

En nuestro caso la variable `x` no cumple con con ninguno de los 4 puntos anteriores. ¿Qué está pasando?



In [92]:
def primera(x):
    print ("Dentro de primera")
    print ("locals(): ", locals())
    def segunda(y):
        print ("Dentro de segunda")
        print ("locals():", locals())
        return x + y
    return segunda

Según lo que hemos hablado, `x` debería ser devuelva por `locals()` de `crea_mi funcion` y no por `locals()` de `mi funcion`, así que probémoslo:

In [93]:
f = primera(1)

Dentro de primera
locals():  {'x': 1}


In [94]:
f(2)

Dentro de segunda
locals(): {'y': 2, 'x': 1}


3

Ohhhhhh, `x` aparece dentro de `locals()` de `segunda`, por eso no me lanza la excepción. Bueno, veamos qué dice la [documentación](https://docs.python.org/2/library/functions.html#locals) acerca de `locals()`:

*Update and return a dictionary representing the current local symbol table. **Free variables are returned by locals() when it is called in function blocks**, but not in class blocks.*
**Note** *The contents of this dictionary should not be modified; changes may not affect the values of local and free variables used by the interpreter.*

¿Qué es una variable libre? Veamos en la [documentación](https://docs.python.org/2/reference/executionmodel.html)

*When a name is used in a code block, it is resolved using the nearest enclosing scope. The set of all such scopes visible to a code block is called the block’s environment.*

*If a name is bound in a block, it is a local variable of that block. If a name is bound at the module level, it is a global variable. (The variables of the module code block are local and global.) **If a variable is used in a code block but not defined there, it is a free variable.***

En nuestro caso `x` es una variable libre porque se define en `primera` y se usa en `segunda`. 

Decimos por tanto que **una función que usa una variable libre es un cierre**.


Por lo tanto **`locals()` devuelve tanto las variables locales y las variables libres cuando se llama desde una función.**


___
###5 propiedades de los cierres

1 El entorno del cierre es almacenado en la propiedad ``__closure__`` (func_closure para Python2) de la función: Las variables libres se almacen ahí en el caso de que se usen [(closure)](https://docs.python.org/3/reference/datamodel.html#the-standard-type-hierarchy)[(cell)](https://docs.python.org/2/c-api/cell.html).

In [86]:
o = 100
def primera(x):
    def segunda(y):
        a = 10
        def tercera(z):
            print("Locals ", locals())
            print("Vars ", vars())
            return a + x + y + z
        return tercera
    return segunda

print(primera(1)(2)(3))

Locals  {'y': 2, 'a': 10, 'z': 3, 'x': 1}
Vars  {'y': 2, 'a': 10, 'z': 3, 'x': 1}
16


In [87]:
print("Variables libres de primera: ", primera.__closure__)

s = primera(1)
print("Variables libres de segunda: ", s.__closure__)
print([i.cell_contents for i in s.__closure__])

t = media(2)
print("Variables libres de tercera: ", t.__closure__)
print([i.cell_contents for i in t.__closure__])


Variables libres de primera:  None
Variables libres de segunda:  (<cell at 0x7f1cfc368cd8: int object at 0x7f1d0506d080>,)
[1]
Variables libres de tercera:  (<cell at 0x7f1cfc368fa8: int object at 0x7f1d0506d1a0>, <cell at 0x7f1cfc368e88: int object at 0x7f1d0506d080>, <cell at 0x7f1cfc368fd8: int object at 0x7f1d0506d0a0>)
[10, 1, 2]


2 Funciones sin variables libres no tienen cierre

In [54]:
def primera(x):
    def segunda(y):
        pass
    return segunda

s = primera(1)
print("Variables libres de segunda: ", s.__closure__)


Variables libres de segunda:  None


3 Se produce un cierre cuando es una función anidada, aunque no utilice variables libres

In [69]:
def primera(w):
    def segunda(x): #no usa variables libres
        def tercera(y): #no usa variables libres
            def cuarta(z): #sí usa variables libres
                return x
            return cuarta
        return tercera
    return segunda

s = primera(1)
print("Variables libres de segunda: ", s.__closure__)
t = s(2)
print("Variables libres de tercera: ", t.__closure__)
c = t(3)
print("Variables libres de cuarta: ", c.__closure__)

Variables libres de segunda:  None
Variables libres de tercera:  (<cell at 0x7f1cfc368df8: int object at 0x7f1d0506d0a0>,)
Variables libres de cuarta:  (<cell at 0x7f1cfc368df8: int object at 0x7f1d0506d0a0>,)


4 Las funciones globales no almacenan nada en `__closure__`

In [70]:
variable_global = 10
def funcion():
    print("A través de globals(): ", globals()["variable_global"])
    print("Directamente: ", variable_global)

funcion()
print ("Variables libres: ",funcion.__closure__)

A través de globals():  10
Directamente:  10
Variables libres:  None


5 Por defecto la asignación crea una variable local. 

En el siguiente código: ¿qué sucede si quiero modificar el valor de la variable `n1` que está declarada en `primera` desde `segunda`?

In [82]:
g = 10
print("valor de g fuera inicio:", g)
def primera(x):
    g = 100
    print("valor de g dentro:", g)
    
    n1 = 1
    print("valor de n1 en primera antes:", n1)
    
    def segunda(y):
        n1 = 0
        print("valor de n1 en segunda:", n1)
        n2 = 2
        
    segunda(2)
    print("valor de n1 en primera despues:", n1)
    return segunda

primera(1)(2)
print("valor de g fuera final:", g)
        

valor de g fuera inicio: 10
valor de g dentro: 100
valor de n1 en primera antes: 1
valor de n1 en segunda: 0
valor de n1 en primera despues: 1
valor de n1 en segunda: 0
valor de g fuera final: 10


Para poder modificar el valor de una variable libre desde la función interna es necesario utilizar `nonlocal`

In [85]:
g = 10
print("valor de g fuera inicio:", g)
def primera(x):
    global g
    g = 100
    print("valor de g dentro:", g)
    
    n1 = 1
    print("valor de n1 en primera antes:", n1)
    
    def segunda(y):
        nonlocal n1
        n1 = 0
        print("valor de n1 en segunda:", n1)
        n2 = 2
        
    segunda(2)
    print("valor de n1 en primera despues:", n1)
    return segunda

primera(1)(2)
print("valor de g fuera final:", g)

valor de g fuera inicio: 10
valor de g dentro: 100
valor de n1 en primera antes: 1
valor de n1 en segunda: 0
valor de n1 en primera despues: 0
valor de n1 en segunda: 0
valor de g fuera final: 100


###¿Para qué pueden servir los cierres?

Se puede utilizar para evitar valores globales u ocultar datos. Son muy utilizados por los **decoradores**...

Algunos recursos utilizados:

    http://mathamy.com/python-closures-and-free-variables.html
    https://gist.github.com/DmitrySoshnikov/700292
    http://www.programiz.com/python-programming/closure
    http://elclubdelautodidacta.es/wp/2013/01/python-el-calificador-de-ambito-nonlocal/
    http://www.kirupa.com/html5/closures_in_javascript.htm
        